In [2]:
# # StratifiedKfold 사용 코드
# import os
# import json
# import numpy as np
# import pandas as pd
# import argparse
# from tqdm import tqdm
# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
# # image
# data_path = '/opt/ml/dataset'
# annotations_path = os.path.join('/opt/ml/dataset/train.json')
# kfold = 5
# # save
# path = '/opt/ml/dataset/stratified_kfold'
# def main():
#     with open(annotations_path, 'r') as f:
#         train_json = json.loads(f.read())
#         images = train_json['images']
#         categories = train_json['categories']
#         annotations = train_json['annotations']
#     annotations_df = pd.DataFrame.from_dict(annotations)
#     x = images
#     y = [[0] * len(categories) for _ in range(len(images))]
#     for anno in annotations:
#         y[anno['image_id']][anno['category_id']] += 1
#     mskf = MultilabelStratifiedKFold(n_splits=kfold, shuffle=True)
#     if not os.path.exists(path):
#         os.mkdir(path)
#     for idx, (train_index, val_index) in tqdm(enumerate(mskf.split(x, y)), total=kfold):
#         train_dict = dict()
#         val_dict = dict()
#         for i in ['info', 'licenses', 'categories']:
#             train_dict[i] = train_json[i]
#             val_dict[i] = train_json[i]
#         train_dict['images'] = np.array(images)[train_index].tolist()
#         val_dict['images'] = np.array(images)[val_index].tolist()
#         train_dict['annotations'] = annotations_df[annotations_df['image_id'].isin(train_index)].to_dict('records')
#         val_dict['annotations'] = annotations_df[annotations_df['image_id'].isin(val_index)].to_dict('records')
#         train_dir = os.path.join(path, f'train_{idx + 1}.json')
#         val_dir = os.path.join(path, f'val_{idx + 1}.json')
#         with open(train_dir, 'w') as train_file:
#             json.dump(train_dict, train_file)
#         with open(val_dir, 'w') as val_file:
#             json.dump(val_dict, val_file)
# main()

100%|██████████| 5/5 [00:03<00:00,  1.43it/s]


In [3]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device

import wandb
import datetime
import torch
import pytz
wandb.login()

/opt/conda/envs/detection/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '
wandb: Currently logged in as: ohsy0512 (cv-06). Use `wandb login --relogin` to force relogin


True

In [2]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
# cfg = Config.fromfile('/opt/ml/code/baseline/mmdetection/configs/yolox/yolox_l_8x8_300e_coco.py')
cfg = Config.fromfile('/opt/ml/code/baseline/UniverseNet/configs/universenet/universenet101_2008d_fp16_4x4_mstrain_480_960_20e_coco.py')
cfg.load_from = 'https://github.com/shinya7y/UniverseNet/releases/download/20.10/universenet101_2008d_fp16_4x4_mstrain_480_960_20e_coco_20201023_epoch_20-3e0d236a.pth'
# cfg.load_from = '/opt/ml/code/baseline/UniverseNet/work_dirs/2023_05_17_18_36_39_GFL/latest.pth'
# 모델 학습
kst = pytz.timezone('Asia/Seoul')
now = datetime.datetime.now(kst)
save_name = now.strftime('%Y_%m_%d_%H_%M_%S_') + cfg.model.type

root='/opt/ml/'

train_file = ####
val_file = ####

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root + '/dataset'
cfg.data.train.ann_file = root + train_file # train json 정보
# cfg.data.train.pipeline[2]['img_scale'] = [(1024,512), (1024, 1200)] # Resize

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root + '/dataset'
cfg.data.val.ann_file = root + val_file # test json 정보
# cfg.data.val.pipeline[1]['img_scale'] = (1024,1024) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root + 'dataset'
cfg.data.test.ann_file = root + 'dataset/test.json' # test json 정보
# cfg.data.test.pipeline[1]['img_scale'] = (1024,1024) # Resize

cfg.data.samples_per_gpu = 16
cfg.data.workers_per_gpu = 1
cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/' + save_name


cfg.model.bbox_head.num_classes = 10

cfg.runner.max_epochs = 50
cfg.optimizer.lr = 0.01
cfg.model.test_cfg.score_thr = 0.05

## wandb 설정
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='MMDetWandbHook',
    # dict(type='WandbLoggerHook',
         init_kwargs={
             'name': save_name,
             'project': 'Betty',
             'entity': 'CV-06',
             'config': {'batch_size':cfg.data.samples_per_gpu,
                        'epochs':cfg.runner.max_epochs}
         },
         interval=5,
         log_checkpoint=True,
         log_checkpoint_metadata=True,
         num_eval_images=100,
         bbox_score_thr=0.05
         )]

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
# cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=5)
cfg.device = get_device()

In [3]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.27s)
creating index...
index created!


In [4]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 4883, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3965  | 1 [Paper]     | 6352  | 2 [Paper pack]  | 897   | 3 [Metal]   | 936   | 4 [Glass]    | 982   |
| 5 [Plastic]       | 2943  | 6 [Styrofoam] | 1263  | 7 [Plastic bag] | 5178  | 8 [Battery] | 159   | 9 [Clothing] | 468   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [5]:
# import logging, sys
# logging.disable(sys.maxsize)
# import warnings
# warnings.filterwarnings(action='ignore')

In [6]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

/opt/ml/code/baseline/UniverseNet/mmdet/models/dense_heads/anchor_head.py:116: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '
2023-05-18 01:17:38,381 - mmcv - INFO - initialize Res2Net with init_cfg {'type': 'Pretrained', 'checkpoint': 'open-mmlab://res2net101_v1d_26w_4s'}
2023-05-18 01:17:38,383 - mmcv - INFO - load model from: open-mmlab://res2net101_v1d_26w_4s
2023-05-18 01:17:38,383 - mmcv - INFO - load checkpoint from openmmlab path: open-mmlab://res2net101_v1d_26w_4s
2023-05-18 01:17:38,656 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

missing keys in source state_dict: layer2.0.convs.0.conv_offset.weight, layer2.0.convs.0.conv_offset.bias, layer2.0.convs.1.conv_offset.weight, layer2.0.convs.1.conv_offset.bias, layer2.0.convs.2.conv_offset.weight, layer2.0.convs.

In [7]:
#### torch.cuda.memory_summary(device=cfg.device, abbreviated=False)
train_detector(model, datasets[0], cfg, distributed=False, validate=False, meta=dict(cfg))
wandb.finish()

2023-05-18 01:17:39,442 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2023-05-18 01:17:42,609 - mmdet - INFO - load checkpoint from local path: /opt/ml/code/baseline/UniverseNet/work_dirs/2023_05_17_18_36_39_GFL/latest.pth
2023-05-18 01:17:42,971 - root - INFO - DeformConv2dPack backbone.layer2.0.convs.0 is upgraded to version 2.
2023-05-18 01:17:42,972 - root - INFO - DeformConv2dPack backbone.layer2.0.convs.1 is upgraded to version 2.
2023-05-18 01:17:42,974 - root - INFO - DeformConv2dPack backbone.layer2.0.convs.2 is upgraded to version 2.
2023-05-18 01:17:42,978 - root - INFO - DeformConv2dPack backbone.layer2.1.convs.0 is upgraded to version 2.
2023-05-18 01:17:42,979 - root - INFO - DeformConv2dPack backbone.layer2.1.convs.1 is upgraded to version 2.
2023-05-18 01:17:42,981 - root - INFO - DeformConv2dPack backbone.layer2.1.convs.2 is upgraded to version 2.
2023-05-18 01:17:42,985 - root - INFO - DeformConv2dPack backbone.layer2.2.convs.0 is upgrade

2023-05-18 01:17:51,634 - mmdet - WARNING - To log evaluation or checkpoint metadata in MMDetWandbHook, `EvalHook` or `DistEvalHook` in mmdet is required, please check whether the validation is enabled.
2023-05-18 01:20:54,542 - mmdet - INFO - Epoch [1][50/306]	lr: 8.000e-05, eta: 15:29:43, time: 3.658, data_time: 0.100, memory: 22548, loss_cls: 0.1342, loss_bbox: 0.1403, loss_dfl: 0.1590, loss: 0.4334, grad_norm: 2.8609
2023-05-18 01:23:53,181 - mmdet - INFO - Epoch [1][100/306]	lr: 8.000e-05, eta: 15:15:53, time: 3.573, data_time: 0.042, memory: 22548, loss_cls: 0.1288, loss_bbox: 0.1311, loss_dfl: 0.1571, loss: 0.4170, grad_norm: 2.7630
2023-05-18 01:26:52,237 - mmdet - INFO - Epoch [1][150/306]	lr: 8.000e-05, eta: 15:09:59, time: 3.581, data_time: 0.042, memory: 22548, loss_cls: 0.1392, loss_bbox: 0.1340, loss_dfl: 0.1584, loss: 0.4317, grad_norm: 3.1552
Thread SenderThread:
Traceback (most recent call last):
  File "/opt/conda/envs/detection/lib/python3.7/site-packages/wandb/sdk/i

BrokenPipeError: [Errno 32] Broken pipe

Error in callback <function _WandbInit._pause_backend at 0x7f709bceecb0> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
import torch
torch.cuda.empty_cache()
import gc
gc.collect()